<a href="https://colab.research.google.com/github/David-Sanchez-Perez/03MAIR---Algoritmos-de-optimizacion---2019/blob/master/Plantilla_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: David Sánchez Pérez  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema: Organizar los horarios de los partidos de La Liga
> 1. Elección de grupos de población homogéneos <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos
de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice
la audiencia.
• Los horarios disponibles se conocen a priori y son los siguientes:

Viernes 20

Sábado 12,16,18,20

Domingo 12,16,18,20

Lunes 20

En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores que tienen relación directa con la audiencia. Hay 4 equipos en la
categoría A, 10 equipos de categoría B y 6 equipos de categoría C.
• Se conoce estadisticamente la audiencia que genera cada partido según los equipos
que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

Categoría A Categoría B Categoría C

Categoría A 2 Millones 1,3 Millones 1 Millones

Categoría B 0.9 Millones 0.75 Millones

Categoría C 0.47 Millones

Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce
según los coeficientes de la siguiente tabla
• Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes
Viernes Sábado Domingo Lunes

12h - 0.55 0.45 -

16h - 0.7 0.75 -

18h - 0.8 0.5 -

20h 0.4 1 1 0.4

Es posible la coincidencia de horarios pero en este
caso la audiencia de cada partido se verá afectada y
se estima que se reduce en porcentaje según la
siguiente tabla dependiendo del número de
coincidencias:

Coincidencias -%

0 0%

1 25%

2 45%

3 60%

4 70%

5 75%

6 78%

7 80%

8 80%



(*) La respuesta es obligatoria


Primero de todo vamos a guardar toda esta información en DataFrames
                                        

In [0]:
# librerías que vamos a utilizar
import pandas as pd
import numpy as np

In [4]:
#Creamos un diccionario a partir de las categorías de los equipos y la audiencia que va a tener
audiencia = {'AA':2e6,'AB':1.3e6,'AC':1e6,'BB':0.9e6,'BC':0.75e6,'CC':0.47e6} #Diccionario audiencia
audiencia #Mostramos el diccionario

{'AA': 2000000.0,
 'AB': 1300000.0,
 'AC': 1000000.0,
 'BB': 900000.0,
 'BC': 750000.0,
 'CC': 470000.0}

In [5]:
horarios = pd.DataFrame(columns=('dia','coef')) #Generamos un Dataframe con los horarios y el coeficiente de publico aplicado

horarios.loc[0]=['viernes_20h',0.4] #Rellenamos el dataframe
horarios.loc[1]=['sabado_12h',0.55]
horarios.loc[2]=['sabado_16h',0.7]
horarios.loc[3]=['sabado_18h',0.8]
horarios.loc[4]=['sabado_20h',1]
horarios.loc[5]=['domingo_12h',0.45]
horarios.loc[6]=['domingo_16h',0.75]
horarios.loc[7]=['domingo_18h',0.85]
horarios.loc[8]=['domingo_20h',1]
horarios.loc[9]=['lunes_20h',0.4]

horarios

,dia,coef
0,viernes_20h,0.40
1,sabado_12h,0.55
2,sabado_16h,0.70
3,sabado_18h,0.80
4,sabado_20h,1.00
5,domingo_12h,0.45
6,domingo_16h,0.75
7,domingo_18h,0.85
8,domingo_20h,1.00
9,lunes_20h,0.40


In [6]:
equipos = pd.DataFrame(columns=('nombre','categoria')) #Ahora generaremos un DataFrame con los equipos y su categoría

equipos.loc[0]=['Celta','B'] #Rellenamos el dataframe
equipos.loc[1]=['Real Madrid','A']
equipos.loc[2]=['Valencia','B']
equipos.loc[3]=['R.Sociedad','A']
equipos.loc[4]=['Mallorca','C']
equipos.loc[5]=['Eibar','C']
equipos.loc[6]=['Athletic','B']
equipos.loc[7]=['Barcelona','A']
equipos.loc[8]=['Leganés','C']
equipos.loc[9]=['Osasuna','C']
equipos.loc[10]=['Villareal','B']
equipos.loc[11]=['Granada','C']
equipos.loc[12]=['Alaves','B']
equipos.loc[13]=['Levante','B']
equipos.loc[14]=['Espanyol','B']
equipos.loc[15]=['Sevilla','B']
equipos.loc[16]=['Betis','B']
equipos.loc[17]=['Valladolid','C']
equipos.loc[18]=['Atlético','B']
equipos.loc[19]=['Getafe','B']

equipos

,nombre,categoria
0,Celta,B
1,Real Madrid,A
2,Valencia,B
3,R.Sociedad,A
4,Mallorca,C
5,Eibar,C
6,Athletic,B
7,Barcelona,A
8,Leganés,C
9,Osasuna,C


In [7]:
#Creamos una función para crear una jornada aleatoria
def aleatorio(equipos):
    df = equipos.iloc[np.random.permutation(len(equipos))] #ordenacion aleatoriaa través del módulo de numpy random permutation
    return df
jornada_aleatoria = aleatorio(equipos) #Llamada a función
jornada_aleatoria

,nombre,categoria
8,Leganés,C
17,Valladolid,C
18,Atlético,B
10,Villareal,B
14,Espanyol,B
2,Valencia,B
19,Getafe,B
12,Alaves,B
15,Sevilla,B
1,Real Madrid,A


In [8]:
#Creamos la tabla de enfrentamientos de la jornada
def enfrentamientos(original):
    df_jornada = pd.DataFrame(columns=('categoria','publico','equipos')) #Creamos otro dataframe para guardar la categoría de ambos partidos, el público y los equipos
    #Luego lo concatenaremos con el horario
    for i in range(0,len(original),2): #Recorremos el dataframe de la jornada aleatoria de dos en dos ya que los partidos son 1 vs 1
        valor = original.iloc(axis=0)[i]['categoria'] #Guardamos el valor de categoría del equipo 1                        
        valor_1 = original.iloc(axis=0)[i+1]['categoria'] #Guardamos el valor de categoría del equipo 2
        if valor < valor_1: #Si el equipo 1 tiene menor categoría que el equipo 2
            cat = valor + valor_1 #Sumamos la categoría
            aud= audiencia[cat] #el público total viene definido en el diccionario audiencia que creamos antes
            partido = original.iloc(axis=0)[i]['nombre'] + '  vs  '+ original.iloc(axis=0)[i+1]['nombre']# columna para poner los partidos
        else:#Si el equipo 1 tiene mayor categoría que el equipo 2
            cat = valor_1 + valor # Hacemos lo mismo que anteriormente
            aud= audiencia[cat]
            partido = original.iloc(axis=0)[i+1]['nombre'] + ' vs '+ original.iloc(axis=0)[i]['nombre']

        # Rellenamos el dataframe con las filas generadas
        df_jornada.loc[len(df_jornada)]=[cat,aud,partido]
    
    return df_jornada
        
jornada = enfrentamientos(jornada_aleatoria) #Llamada a función
jornada

,categoria,publico,equipos
0,CC,470000.0,Valladolid vs Leganés
1,BB,900000.0,Villareal vs Atlético
2,BB,900000.0,Valencia vs Espanyol
3,BB,900000.0,Alaves vs Getafe
4,AB,1300000.0,Real Madrid vs Sevilla
5,BC,750000.0,Betis vs Eibar
6,AC,1000000.0,R.Sociedad vs Osasuna
7,AB,1300000.0,Barcelona vs Athletic
8,BC,750000.0,Celta vs Granada
9,BC,750000.0,Levante vs Mallorca


#**(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>**


#¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones. 




**Respuesta**

Sin tener en cuenta las restricciones, al haber 20 equipos significa que se jugarán 10 partidos, por lo tanto las posibilidades son 10.

En cambio, con restricciones, para resolver este problema tenemos que tener en cuenta dos cosas:
    - coeficientes de equipos con mayor categoría
    - coeficientes de los horarios con mejores audiencias
Por lo tanto para resolver este problema, simplemente hay que ordenar estos dos coeficientes de mayor a menos y así obtener la mayor audiencia posible. Una parte importante a tener en cuenta es que descartamos la posibilidad de repetir horario, ya que como el partido con mayor categoría va a ser el que esté colocado en mejor horario, si ponemos otro partido a la vez nos quitaría mucha audiencia.

Por lo tanto, para resolver este problema vamos a realizar un algoritmo de ordenación de coeficientes, el cuál en el peor de los casos va a ser O(n^2). El número posible de soluciones es N!

#**Modelo para el espacio de soluciones<br>**
#(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


**Respuesta**

Como vamos a tener que almacenar datos en un DataFrame vamos a utilizar Pandas y para realizar operaciones matemáticas utilizaremos Numpy

#**Según el modelo para el espacio de soluciones<br>**
# (*)¿Cual es la función objetivo?
# (*)¿Es un problema de maximización o minimización?

**Respuesta**

La función objetivo es de tipo lineal:
f(x1,...,xn)= h1·x1 + ... + hn·xn
Siendo x las audiencias de los partidos según la categoría y h el coeficiente según el horario

El problema es de maximicación porque tal y como se ha explicado en el apartado anterior, se busca que la audiencia de cada jornada sea máxima

# Diseña un algoritmo para resolver el problema por fuerza bruta

**Respuesta**

In [9]:
def orden(df,col,boolean): #Función para ordenar por fuerza bruta la categoría y los horarios
    df_copia = df.copy() #Generamos un dataframe copia

    for i in range(len(df)): #Recorremos el data frame en busca del valor 
        val_min = i # Almacenaremos el indice del menor valor
    
        for j in range(i+1,len(df)): #Para comparar el primer valor vs el resto
            if (boolean==1): #Para ordenar los enfrentamientos de la jornada
                if (df_copia.iloc(axis=0)[val_min][col] > df_copia.iloc(axis=0)[j][col]): #Si hay un valor con mejor categoría
                    val_min = j #Guardamos la posición del valor mayor para colocarlo al principio
            else: #Para ordenar los coeficientes de los horarios
                if (df_copia.iloc(axis=0)[val_min][col] < df_copia.iloc(axis=0)[j][col]): #Hacemos lo mismo que antes
                    val_min = j 
   
        actual = df_copia.iloc(axis=0)[i].copy() #Copiamos el valor de la fila i (la primera)
    
        df_copia.loc[i] = df_copia.iloc(axis=0)[val_min] #Insertamos los valores de la fila con mayor categoría encontrado en la posición i

        df_copia.loc[val_min] = actual #Los valores que ocupaban la fila i pasan al resto de filas por comparar

    return df_copia

equipos_ord = orden(jornada,'categoria',1) #Ordenamos los equipos por categorías
coef_ord =orden(horarios, 'coef',0) #Ordenamos los horarios por coeficientes

equipos_ord[['dia','coef']] = coef_ord #Concatenamos las tablas
equipos_ord

,categoria,publico,equipos,dia,coef
0,AB,1300000.0,Real Madrid vs Sevilla,sabado_20h,1.00
1,AB,1300000.0,Barcelona vs Athletic,domingo_20h,1.00
2,AC,1000000.0,R.Sociedad vs Osasuna,domingo_18h,0.85
3,BB,900000.0,Alaves vs Getafe,sabado_18h,0.80
4,BB,900000.0,Valencia vs Espanyol,domingo_16h,0.75
5,BB,900000.0,Villareal vs Atlético,sabado_16h,0.70
6,BC,750000.0,Betis vs Eibar,sabado_12h,0.55
7,BC,750000.0,Celta vs Granada,domingo_12h,0.45
8,BC,750000.0,Levante vs Mallorca,viernes_20h,0.40
9,CC,470000.0,Valladolid vs Leganés,lunes_20h,0.40


# Calcula la complejidad del algoritmo por fuerza bruta

**Respuesta**

La complejidad es O(n^2) ya que tenemos dos bucles for

# (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

**Respuesta**

Un algoritmo que mejora la complejidad de la fuerza bruta y que además hemos aplicado en clase es el algoritmo de busqueda quick short.

In [0]:
def quick_sort(df, col):  #Algoritmo quicksort visto en clase en la AG1 pero con la modificación de no usar un list al final

    if len(df) == 1: #Cuando solo quede un valor a la izq devolveremos el valor, ya que significa que es el más grande
        return df
  
    elif len(df) == 2: #Cuando queden dos valores a la izq devolveremos el máximo de ambos primero y luego el menor
    
        if df.iloc(axis=0)[0][col] > df.iloc(axis=0)[1][col]: #Si es mayor
            return df
        else:
            return pd.DataFrame([df.iloc(axis=0)[1], df.iloc(axis=0)[0]])
    
    else: #Si el df contiene más de dos valores
        izq = pd.DataFrame(columns=df.columns) #Creamos el df izq
        der = pd.DataFrame(columns=df.columns) #Creamos el df der

        pivote = (df.loc(axis =1)[col].sum())/len(df) # Calculamos el pivote haciendo la media de la columna a ordenar

        for i in range(len(df)): #Recorremos la columna

            if df.iloc(axis=0)[i][col] >= pivote: #Si es mayor o igual que el pivote lo mandamos a la izquierda
                izq = izq.append(df.iloc(axis=0)[i]) #Lo añadimos a la izquierda
        
            else: #Si es menor que el pivote lo mandamos a la derecha
                der = der.append(df.iloc(axis=0)[i])
        

        if len(izq)!=0 and len(der)!=0: #Mientras haya valores por ordenar a izquierda o derecha haremos una recursividad para ir ordenandolos cuantas veces sea necesario
      
            df_I = quick_sort(izq,col) #ordenamos los valores a la izquierda
            df_D = quick_sort(der,col) #ordenamos los valores a la derecha
      
            return  df_I.append(df_D) #devolvemos el df concatenado de izquierda a derecha
    
        else:
            return df
        
coef_ord = quick_sort(horarios,'coef') #Ordenamos los coeficientes de los horarios
equipos_ord = quick_sort(jornada,'publico') #Ordenamos las categorías de los partidos

equipos_ord.index = coef_ord.index #Igualamos los indices para concatenarlos

equipos_ord[['dia','coef']] = coef_ord #Concatenamos las tablas
equipos_ord

# (*)Calcula la complejidad del algoritmo 

**Respuesta**

La complejidad del algoritmo quickshort es de O(n·log(n)) tal y como vimos en clase

# Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

**Respuesta**

In [10]:
#Tal como hemos hecho anteriormente
jornada_aleatoria = aleatorio(equipos)
jornada = enfrentamientos(jornada_aleatoria)
jornada

,categoria,publico,equipos
0,BB,900000.0,Villareal vs Levante
1,CC,470000.0,Granada vs Valladolid
2,BB,900000.0,Alaves vs Celta
3,BC,750000.0,Atlético vs Leganés
4,BB,900000.0,Espanyol vs Sevilla
5,AB,1300000.0,Barcelona vs Athletic
6,AB,1300000.0,Real Madrid vs Valencia
7,AB,1300000.0,R.Sociedad vs Betis
8,BC,750000.0,Getafe vs Osasuna
9,CC,470000.0,Mallorca vs Eibar


# Aplica el algoritmo al juego de datos generado

**Respuesta**

Ya se ha resuelto tanto en el algoritmo de fuerza bruta como en el de quick short

# Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

**Respuesta**

Las únicas referencias utilizadas han sido los apuntes de la asignatura de python para numpy y manipular los dataframes y los apuntes de esta asignatura de algoritmos de optimización para realizar el algoritmo por fuerza bruta y quick short

# Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

**Respuesta**

Una buena mejora sería hacer un algoritmo de ramificación y poda para eliminar opciones, como por ejemplo los equipos con categoría C, en principio no deberían jugar en horarios con alto público. Lo mismo para equipos con categoría A y horarios con poco público.